# Deep Learning Project - Lisbon traffic sign experiment

## <span style="color:orange">This notebook's purpose is to implement an experiment for classifying photographies of traffic signs taken in the city of Lisbon and to check how well the implemented traffic sign classification model performs on these images.</span>

## <span style="color:orange">Please adjust paths before running.</span>

In [1]:
import numpy as np
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import os
import matplotlib.pyplot as plt

## Predicting test set images

In [2]:
# Load a saved model
model = keras.models.load_model('model_final_without_weights_after_epoch_15')

In [3]:
test_dir = '/Users/philippmetzger/Documents/GitHub/Deep_Learning_Project_Group_10/Data/lisbon_traffic_signs'
#test_dir = '/Users/philippmetzger/Documents/GitHub/Deep_Learning_Project_Group_10/Data/Meta'

### Load ground truth data

In [4]:
gt = pd.read_excel('/Users/philippmetzger/Documents/GitHub/Deep_Learning_Project_Group_10/Data/lisbon_traffic_signs/ground_truth.xlsx')
gt

,Index,Target
0,1,17
1,2,17
2,3,17
3,4,17
4,5,17
...,...,...
81,106,17
82,107,17
83,108,17
84,110,35


### Normalization

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    batch_size=20,
    shuffle=False,
    #class_mode=None,
    class_mode="categorical",
    color_mode="grayscale"
    )

x_test=np.concatenate([test_generator.next()[0] for i in range(test_generator.__len__())])

test_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

test_datagen.fit(x_test)

Found 86 images belonging to 1 classes.


##### Re-Creating the generators with the normalized image data

In [6]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(100, 100),
        batch_size=20,
        shuffle=False,
        color_mode="grayscale",
        class_mode=None)

test_generator.reset()
pred_scores = model.predict_generator(test_generator)
class_pred = np.argmax(pred_scores, axis=-1)

Found 86 images belonging to 1 classes.


/Users/philippmetzger/anaconda3/envs/dl_pip_2/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [7]:
target = list(gt['Target'])
index = list(gt['Index'])

In [8]:
df = pd.DataFrame(data={'index': index, 'target':target, 'prediction':class_pred})
df

,index,target,prediction
0,1,17,17
1,2,17,17
2,3,17,17
3,4,17,17
4,5,17,17
...,...,...,...
81,106,17,17
82,107,17,17
83,108,17,17
84,110,35,35


In [9]:
i = 0

slice = [range(0,40), range(40,80), range(80,86)]

df.iloc[slice[i],:]

,index,target,prediction
0,1,17,17
1,2,17,17
2,3,17,17
3,4,17,17
4,5,17,17
5,6,1,1
6,7,1,1
7,8,1,1
8,9,1,1
9,10,1,1


In [10]:
i = 1

df.iloc[slice[i],:]

,index,target,prediction
40,58,13,13
41,59,33,34
42,60,13,13
43,61,17,17
44,62,16,16
45,63,17,17
46,64,15,13
47,65,17,17
48,66,13,13
49,67,14,14


In [11]:
i = 2

df.iloc[slice[i],:]

,index,target,prediction
80,102,13,13
81,106,17,17
82,107,17,17
83,108,17,17
84,110,35,35
85,112,1,1


In [12]:
bool_ = (class_pred == target)
bool_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False, False,
       False,  True,  True,  True,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True, False, False,  True, False,  True, False,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True])

In [13]:
sum(bool_) / len(bool_)

0.8023255813953488

In [16]:
sum(bool_)

69

In [15]:
len(bool_)

86

In [14]:
df.to_excel('Lisbon_traffic_signs_results.xlsx')